In [1]:
# 利用卷积神经网络识别MNIST数字图像
# 实例化一个小型的卷积神经网络
# 它是 Conv2D 层和 MaxPooling2D 层的堆叠。

from keras import layers
from keras import models

model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

model.summary()# 目前神经网络的架构
#可以看到模型各个层的组成（dense表示全连接层）。也能看到数据经过每个层后，输出的数据维度。还能看到Param，它表示每个层参数的个数
#每个 Conv2D 层和 MaxPooling2D 层的输出都是一个形状为 (height, width,channels) 的 3D 张量。
#宽度和高度两个维度的尺寸通常会随着网络加深而变小。
#通道数量由传入 Conv2D 层的第一个参数所控制（32 或 64）。

#问题？？？
#1.Outputshape是如何计算的？
#2.Param是如何计算的？

Using TensorFlow backend.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [2]:
#在卷积神经网络上添加分类器
model.add(layers.Flatten())#Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。Flatten不影响batch的大小
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))
#手写数字识别将进行 10 类别分类，最后一层使用带 10 个输出的 softmax 激活。
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [3]:
def load_data():
    path=r'D:\mnist.npz'# 放置mnist.npz的目录
    f=np.load(path)
    x_train,y_train=f['x_train'],f['y_train']
    x_test,y_test=f['x_test'],f['y_test']
    f.close()
    return (x_train,y_train),(x_test,y_test)

In [6]:
import numpy as np
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = load_data()#train_images_shape (60000, 28, 28) train_labels_shape shape (60000,)
# 对于每个训练示例，我们的神经网络都将采用单个矢量，因此我们需要对输入进行整形，以使每个28x28图像成为单个784维矢量。 
# 我们还将缩放输入范围为[0-1]而不是[0-255]
train_images=train_images.reshape(60000,28,28,1)#conv2d_input to have 4 dimensions,
test_images=test_images.reshape(10000,28,28,1)
train_images=train_images.astype('float32')
test_images=test_images.astype('float32')
train_images/=256# 值在0-255之间，归一化
test_images/=256
# 多类分类问题与二类分类问题类似，需要将类别变量（categorical function）的输出标签转化为数值变量。
# 在多分类问题中我们将转化为虚拟变量（dummy variable）：
# 即用one hot encoding方法将输出标签的向量（vector）转化为只在出现对应标签的那一列为1，其余为0的布尔矩阵。
# 这一步转化工作我们可以利用keras中的np_utils.to_categorical函数来进行。
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5,batch_size=64)


Epoch 1/5
60000/60000 [==============================] - 28s 463us/step - loss: 0.1711 - accuracy: 0.9459
Epoch 2/5
60000/60000 [==============================] - 21s 350us/step - loss: 0.0492 - accuracy: 0.9846
Epoch 3/5
60000/60000 [==============================] - 21s 349us/step - loss: 0.0342 - accuracy: 0.9893
Epoch 4/5
60000/60000 [==============================] - 21s 348us/step - loss: 0.0253 - accuracy: 0.9919
Epoch 5/5
60000/60000 [==============================] - 22s 359us/step - loss: 0.0200 - accuracy: 0.9938


In [7]:
test_loss,test_acc=model.evaluate(test_images,test_labels)

10000/10000 [==============================] - 4s 434us/step


In [8]:
print(test_loss,test_acc)
#第 2 章密集连接网络的测试精度为 97.8%，但这个简单卷积神经网络的测试精度达到了98.9%

0.040397019239835935 0.9889000058174133


In [ ]:
5.1.1　卷积运算

# 密集连接层和卷积层的根本区别在于， Dense 层从输入特征空间中学到的是全局模式

# 卷积神经网络学到的模式有平移不变性
# 卷积神经网络可以学到模式的空间层次结构

# 特征图：对于包含两个空间轴（高度和宽度）和一个深度轴（也叫通道轴）的 3D 张量，其卷积也叫特征图
# 深度轴的不同通道不再像 RGB 输入那样代表特定颜色，而是代表过滤器（filter）。过滤器对输入数据的某一方面进行编码
# 这也是特征图这一术语的含义：深度轴的每个维度都是一个特征（或过滤器），而 2D 张量 output[:, :, n] 是这个过滤器在输入上的响应的二维空间图（map）。

# 卷积由以下两个关键参数所定义。
#  从输入中提取的图块尺寸：这些图块的大小通常是 3×3 或 5×5。本例中为 3×3，这是很常见的选择。
#  输出特征图的深度：卷积所计算的过滤器的数量。本例第一层的深度为 32，最后一层的深度是 64。
# 对于 Keras 的 Conv2D 层：Conv2D(output_depth,(window_height, window_width)) 

"""
1. 理解边界效应与填充 
   如果希望输出特征图的空间维度与输入相同，那么可以使用填充（padding）
   填充是在输入特征图的每一边添加适当数目的行和列，使得每个输入方块都能作为卷积窗口的中心。
   对于 Conv2D 层，可以通过 padding 参数来设置填充，这个参数有两个取值： 
   "valid" 表示不使用填充（只使用有效的窗口位置）； "same" 表示“填充后输出的宽度和高度与输入相同”。
   padding 参数的默认值为 "valid" 。
2. 理解卷积步幅
   对卷积的描述都假设卷积窗口的中心方块都是相邻的。
   但两个连续窗口的距离是卷积的一个参数，叫作步幅，默认值为 1。
   也可以使用步进卷积（strided convolution），即步幅大于 1 的卷积
"""

5.1.2　最大池化运算
# 使用下采样的原因，一是减少需要处理的特征图的元素个数，二是通过让连续卷积层的观察窗口越来越大（即窗口覆盖原始输入的比例越来越大），
# 从而引入空间过滤器的层级结构。

# 最大池化是从输入特征图中提取窗口，并输出每个通道的最大值。
# 它的概念与卷积类似，但是最大池化使用硬编码的 max 张量运算对局部图块进行变换，而不是使用学到的线性变换（卷积核）。
# 最大池化通常使用 2×2 的窗口和步幅 2，其目的是将特征图下采样 2 倍。与此相对的是，卷积通常使用 3×3 窗口和步幅 1。


# 使用下采样的原因，一是减少需要处理的特征图的元素个数，二是通过让连续卷积层的观察窗口越来越大（即窗口覆盖原始输入的比例越来越大），
# 从而引入空间过滤器的层级结构。
